In [ ]:
%config Complete.use_jedi = False
#!pip install datatable
#!pip install mlxtend
#http://rasbt.github.io/mlxtend/user_guide/evaluate/permutation_test/

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import json
import datatable as dt
from mlxtend.evaluate import permutation_test
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
%%time
starbucks = dt.fread("./starbucks_data/starbucks_df_per_person.csv").to_pandas()

In [ ]:
starbucks.info()

In [ ]:
#6 rows are missing in the duration thingy, we will drop these, they should not have that h impact
starbucks = starbucks[starbucks.difficulty_offered.notna() & starbucks.duration_offered.notna()]
starbucks.info()

In [ ]:
#we need that later for mapping
starbucks.to_csv("./starbucks_data/starbucks_df_per_id.csv", index=False)
starbucks.drop(columns=["id"], inplace=True)

In [ ]:
starbucks.gender.unique()
# the empty gender rows may be equal to the missing income rows, we will check that later before the imputation

In [ ]:
fig = px.histogram(starbucks, x="gender")
fig.show()

In [ ]:
fig = px.histogram(starbucks, x="income")
fig.show()

In [ ]:
starbucks_filtered=starbucks[starbucks.gender.notna()]
fig = px.histogram(starbucks_filtered, x="income", color=starbucks_filtered.gender, marginal="violin",
                         hover_data=starbucks.columns)
fig.show()

In [ ]:
#interestingly we see for our data without missing values that we have mor male consumer with less income (right_skewed toward male)
#while the distribution of female and Other seems normally distributed the male's distribution is not
#epecially male with higher income (right side of distribution) do not tend to be in the data so often 
print(starbucks_filtered[(starbucks_filtered.gender =="M") & (starbucks_filtered.income > 100000)].count().max())
print(starbucks_filtered[(starbucks_filtered.gender =="F") & (starbucks_filtered.income > 100000)].count().max())

In [ ]:
#Imputing will be done with ExtraTrees since it is as stable as RandoMForest but much faster and without AWS
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
#no need for train test split, since we will split it with NaNs

# the question is which value we should impute first, so does income prediction benefits from having the gender first or vice versa?
# we do not know if the people who has not answered the income question are also Male, which could lift the distribution to a normally state for example, and without having the income
# the problem is, that these variables are henne-ei problem, because salary sometimes underlies a gender-pay-gap unfortunately, so 
#the gender would have been the ideal variable to forecast income and vice versa

#however I believe that forecasting income first would be more reliable since ppl with less income would probably receive specific rewards, 
#they do not have the money to buy so often, thus they probably get more bogo offers
#thus forecasting income with the coupons could be more effective without distorting the forecast of income when having the gender
#lets check that

In [ ]:
starbucks_filtered=starbucks[starbucks.gender.notna()]
fig = px.histogram(starbucks_filtered, x="income", y=starbucks_filtered.total_reward_completed, color=starbucks_filtered.gender, marginal="violin",
                         hover_data=starbucks.columns)
fig.show()
#since we counted the bogo offers per person several bogos per person may appear, interestingly 
# the bogo offers show what we expected , but it also looks like the MAle distribution

In [ ]:
#why do we look at the median for this? Well splitting income into two halves may be useful, 
#e.g. uci income data also makes a label for above or below 50k (source)
print(np.median(starbucks_filtered.income.fillna(0)))
print(np.mean(starbucks_filtered.income))

In [ ]:
#however the median seems not to help, it seems that they also get less bogo, probably they cant afford to buy one so often, that they should not get the next one for free,
#the strcuture is that low income people get more informational and less value rewarindg coupons, but it seems not significant ()
print("bogo high:", starbucks_filtered.bogo[starbucks_filtered.income > 59000.0].sum())
print("bogo low:", starbucks_filtered.bogo[starbucks_filtered.income <= 59000.0].sum())
print("discount high:", starbucks_filtered.discount[starbucks_filtered.income > 59000.0].sum())
print("discount low:", starbucks_filtered.discount[starbucks_filtered.income <= 59000.0].sum())
print("informational high:", starbucks_filtered.informational[starbucks_filtered.income > 59000.0].sum())
print("informational low:", starbucks_filtered.informational[starbucks_filtered.income <= 59000.0].sum())

#permutation test?

In [ ]:
#however the mean seems not to help, it seems that they also get less bogo, probably they cant afford to buy one so often, that they should not get the next one for free,
#the strcuture is that low income people get more informational and less value rewarindg coupons, but it seems not significant ()
print("bogo high:", starbucks_filtered.bogo[starbucks_filtered.income > 65405].sum())
print("bogo low:", starbucks_filtered.bogo[starbucks_filtered.income <= 65405].sum())
print("discount high:", starbucks_filtered.discount[starbucks_filtered.income > 65405].sum())
print("discount low:", starbucks_filtered.discount[starbucks_filtered.income <= 65405].sum())
print("informational high:", starbucks_filtered.informational[starbucks_filtered.income > 65405].sum())
print("informational low:", starbucks_filtered.informational[starbucks_filtered.income <= 65405].sum())
#the mean strengthens the underlying structure from the median

In [ ]:
#we still try to do a permutation test, labeling ppl in the data set by mean and median and lets see if this would be a valid startegy
#making group_names (permutation mostly works on the mean, so lets see)
starbucks_filtered["income_mean"] = np.where(starbucks_filtered.income >= 65405, "high", "low")
starbucks_filtered["income_median"] = np.where(starbucks_filtered.income >= 59000, "high", "low")

starbucks_bogo_high_mean = starbucks_filtered.bogo[starbucks_filtered.income_mean == "high"]
starbucks_bogo_low_mean = starbucks_filtered.bogo[starbucks_filtered.income_mean == "low"]

starbucks_bogo_high_median = starbucks_filtered.bogo[starbucks_filtered.income_median == "high"]
starbucks_bogo_low_median = starbucks_filtered.bogo[starbucks_filtered.income_median == "low"]

In [ ]:
# real exact permutatuion test would be too much because of all permutations, but we will use a randomization test, paired, to check if 
# an approximate delivers us an insight if both distributions of bogos area equal between different income structures

In [ ]:
for _ in range(5):
    p_value = permutation_test(list(starbucks_bogo_high_mean), list(starbucks_bogo_low_mean),
                               method='approximate',
                               num_rounds=30000,
                               seed=_)
    print(pd.DataFrame([p_value]).round(5))
#Since p-value < alpha 0.05, we can reject the null hypothesis that the two samples come from the same distribution.
# thus the bogos are different between income in terms of mean, lets have a look at the median

In [ ]:
for _ in range(5):
    p_value = permutation_test(list(starbucks_bogo_high_median), list(starbucks_bogo_low_median),
                               method='approximate',
                               num_rounds=30000,
                               seed=_)
    print(pd.DataFrame([p_value]).round(5))
    
#also different in terms of median

In [ ]:
#since the magnitude in differences shows differences regardless of median and mean, we can be assured this feature may some impact in forecastin income
# but we will see this from the permutation importance of a method also
print("mean_differences")
print("****************")
print("bogo diff:", starbucks_filtered.bogo[starbucks_filtered.income > 59000].sum() - starbucks_filtered.bogo[starbucks_filtered.income <= 59000].sum())
print("discount diff:", starbucks_filtered.discount[starbucks_filtered.income > 59000].sum() - starbucks_filtered.discount[starbucks_filtered.income <= 59000].sum())
print("informational diff:", starbucks_filtered.informational[starbucks_filtered.income > 59000].sum() - starbucks_filtered.informational[starbucks_filtered.income <= 59000].sum())

In [ ]:
print("median_differences")
print("****************")
print("bogo diff:", starbucks_filtered.bogo[starbucks_filtered.income > 65405].sum() - starbucks_filtered.bogo[starbucks_filtered.income <= 65405].sum())
print("discount diff:", starbucks_filtered.discount[starbucks_filtered.income > 65405].sum() - starbucks_filtered.discount[starbucks_filtered.income <= 65405].sum())
print("informational diff:", starbucks_filtered.informational[starbucks_filtered.income > 65405].sum() - starbucks_filtered.informational[starbucks_filtered.income <= 65405].sum())

In [ ]:
#are empty gender rows and missing income rows equal?
any(starbucks[starbucks.income.isna()].index == starbucks[starbucks.gender==""].index)

In [ ]:
#ok then we only need one column to filter for both missing features and creating a missing and a full data frame
starbucks_income_gender_missing = starbucks[starbucks.income.isna()]
starbucks_income_gender_full = starbucks[starbucks.income.notna()]

# getting the targets from the full data frame
starbucks_income_full_y = starbucks_income_gender_full.income
starbucks_gender_full_y = starbucks_income_gender_full.gender

#dropping targets from the features
starbucks_income_gender_full_x = starbucks_income_gender_full.drop(["income", "gender"], axis=1)

In [ ]:
#Labelencoding for gender
lbl = LabelEncoder()
lbl.fit(starbucks_gender_full_y)
transformed_gender = lbl.transform(starbucks_gender_full_y)
print(lbl.classes_)
print(transformed_gender)
starbucks_gender_full_y = transformed_gender

In [ ]:
starbucks_gender_full_y

In [ ]:
starbucks_filtered

#0, 1, 0 = Male
#1, 0, 0 F
#other is O

In [ ]:
#Do we need gender in the train set for Nan prediction?
#no not because we dont need it, but because it doesnt work!
#we cant use encoding for gender in train data since our test set also wont have these features for predict, 
#we cant use gender for prediction of income if the test set has NaNs in the gender in the same way as income, even if Nan means empty

In [ ]:
#Making a simple prediction imputation for income, trees need no scaling doesnt matter if reg tree or class tree
# we fit a model on the full data
X_train, X_test, y_train, y_test = train_test_split(starbucks_income_gender_full_x, starbucks_income_full_y, random_state=0)

steps = [('extra', ExtraTreesRegressor())]
pipeline = Pipeline(steps)

params = {
    "extra__max_depth": [4, 8, 16, 24, 32],
    "extra__min_samples_split": [2, 4, 6, 8, 10, 12, 16, 24, 32],
    "extra__min_samples_leaf": [2, 4, 6, 8, 10, 12, 24, 32]
}

random_search = RandomizedSearchCV(pipeline, params, cv = 5, n_iter=50, random_state=0, verbose=5, scoring="neg_mean_absolute_error")
random_search.fit(X_train, y_train)

In [ ]:
#ExtraTreesRegressor Imputing income with best params solution
extraReg = ExtraTreesRegressor(
    min_samples_split=random_search.best_params_["extra__min_samples_split"],
    min_samples_leaf=random_search.best_params_["extra__min_samples_leaf"], 
    max_depth=random_search.best_params_["extra__max_depth"])

extraReg.fit(X_train, y_train)

# train_r2_score partial data
train_r2 = r2_score(np.array(y_train).reshape(-1, 1), extraReg.predict(X_train))
print(train_r2)

#test r2_score partial_data
test_r2 = r2_score(y_test, extraReg.predict(X_test))
print(test_r2)

In [ ]:
#Have a look at our train r2 with full data (retrain on full data after tuning)
extraReg.fit(starbucks_income_gender_full_x, starbucks_income_full_y)
train_r2 = r2_score(starbucks_income_full_y, extraReg.predict(starbucks_income_gender_full_x))
train_r2

In [ ]:
starbucks_income_gender_missing_x = starbucks_income_gender_missing.drop(["income", "gender"], axis=1)
starbucks_income_gender_missing.income = extraReg.predict(starbucks_income_gender_missing_x)

In [ ]:
starbucks_income_gender_missing
# forecasting NaN worked for income

In [ ]:
#Making a simple prediction imputation for gender
X_train, X_test, y_train, y_test = train_test_split(
    starbucks_income_gender_full_x, 
    starbucks_gender_full_y, 
    random_state=0, 
    stratify=starbucks_gender_full_y
)

steps = [('extra', ExtraTreesClassifier())]
pipeline = Pipeline(steps)

params = {
    "extra__max_depth": [4, 8, 16, 24, 32],
    "extra__min_samples_split":[2, 4, 6, 8, 10, 12, 16, 24, 32],
    "extra__min_samples_leaf":[2, 4, 6, 8, 10, 12, 24, 32]
}

random_search = RandomizedSearchCV(pipeline, params, cv = 5, n_iter=50, random_state=0, verbose=5, scoring="accuracy")
random_search.fit(X_train, y_train)

In [ ]:
#LabelBinarizer for gender
from sklearn.preprocessing import LabelBinarizer
lbl = LabelBinarizer()
lbl.fit(starbucks_gender_full_y)
transformed_gender = lbl.transform(starbucks_gender_full_y)
print(lbl.classes_)
print(transformed_gender)
starbucks_gender_full_y = transformed_gender

In [ ]:
starbucks_gender_full_y

In [ ]:
starbucks_gender_full_y

In [ ]:
#ExtraTreesClassifier Imputing gender with best params solution
extraReg_class = ExtraTreesClassifier(
    min_samples_split=random_search.best_params_["extra__min_samples_split"],
    min_samples_leaf=random_search.best_params_["extra__min_samples_leaf"], 
    max_depth=random_search.best_params_["extra__max_depth"])

extraReg_class.fit(X_train, y_train)

# train_r2_score partial data
train_acc = accuracy_score(y_train, extraReg_class.predict(X_train))
print(train_acc)

#test r2_score partial_data
test_acc = accuracy_score(y_test, extraReg_class.predict(X_test))
print(test_acc)

In [ ]:
# retrain fit on full train data and then forecast
extraReg_class.fit(starbucks_income_gender_full_x, starbucks_gender_full_y)
#seems like an overfitted tree?
train_accuracy = accuracy_score(starbucks_gender_full_y, extraReg_class.predict(starbucks_income_gender_full_x))
train_accuracy

In [ ]:
#no predict proba since we want to work directly with imputed labels
starbucks_income_gender_missing.gender = extraReg_class.predict(starbucks_income_gender_missing_x)

In [ ]:
starbucks_income_gender_missing

In [ ]:
starbucks_income_gender_missing.gender.unique()
#should ow be full with forecasted imputed values

In [ ]:
starbucks_income_gender_full.gender.describe()

In [ ]:
(starbucks_income_gender_missing.gender == 0).head(50)

In [ ]:
starbucks_income_gender_missing.gender = np.where(starbucks_income_gender_missing.gender == 0, "M", "F")

In [ ]:
starbucks_income_gender_missing.gender.describe()

In [ ]:
#concatenate persons with missing values with persons and real data
starbucks_imputed_full = starbucks_income_gender_full.append(starbucks_income_gender_missing)

In [ ]:
#previous
fig = px.histogram(starbucks_income_gender_full, x="gender")
fig.show()

In [ ]:
#now
fig = px.histogram(starbucks_imputed_full, x="gender")
fig.show()
#seems more distorted now, however, if we would have filled the most frequent we would have only male

In [ ]:
fig = px.histogram(starbucks_imputed_full, x="income")
fig.show()

In [ ]:
#Now we look at the distributions with the imputed values again
fig = px.histogram(starbucks_imputed_full, x="income", color=starbucks_imputed_full.gender, marginal="violin",
                         hover_data=starbucks_imputed_full.columns)
fig.show()

In [ ]:
starbucks_imputed_full.age.unique()

In [ ]:
starbucks_imputed_full.age = np.where(starbucks_imputed_full.age > 90, 0, starbucks_imputed_full.age)

In [ ]:
#ok then we only need one column to filter for both missing features
starbucks_age_missing = starbucks_imputed_full[starbucks_imputed_full.age == 0]
starbucks_age_full = starbucks_imputed_full[starbucks_imputed_full.age != 0]

In [ ]:
starbucks_age_full_y = starbucks_age_full.age
starbucks_age_full_x = starbucks_age_full.drop(["age"], axis=1)

In [ ]:
starbucks_age_full_x.gender = np.where(starbucks_age_full_x.gender=="M", 0, np.where(starbucks_age_full_x.gender=="F", 1, 2))

In [ ]:
#Making a simple prediction imputation for age, since the values over 90 are bullshit
X_train, X_test, y_train, y_test = train_test_split(starbucks_age_full_x, starbucks_age_full_y, random_state=0)


steps = [('extra', ExtraTreesRegressor())]
pipeline = Pipeline(steps)

params = {
    "extra__max_depth": [4, 8, 16, 24, 32],
    "extra__min_samples_split":[2, 4, 6, 8, 10, 12, 16, 24, 32],
    "extra__min_samples_leaf":[2, 4, 6, 8, 10, 12, 24, 32]
}

random_search = RandomizedSearchCV(
    pipeline, 
    params, 
    cv = 5, 
    n_iter=50, 
    random_state=0,
    verbose=5, 
    scoring="neg_mean_absolute_error"
)

random_search.fit(X_train, y_train)

In [ ]:
#ExtraTreesRegressor Imputing age with best params
extraReg = ExtraTreesRegressor(
    min_samples_split=random_search.best_params_["extra__min_samples_split"],
    min_samples_leaf=random_search.best_params_["extra__min_samples_leaf"], 
    max_depth=random_search.best_params_["extra__max_depth"])

extraReg.fit(X_train, y_train)

# train_r2_score partial data
train_r2 = r2_score(np.array(y_train).reshape(-1, 1), extraReg.predict(X_train))
print(train_r2)

#test r2_score partial_data
test_r2 = r2_score(y_test, extraReg.predict(X_test))
print(test_r2)


In [ ]:
#Have a look at our full train r2
extraReg.fit(starbucks_age_full_x, starbucks_age_full_y)
train_r2 = r2_score(starbucks_age_full_y, extraReg.predict(starbucks_age_full_x))
train_r2

In [ ]:
starbucks_age_missing_x = starbucks_age_missing.drop(["age"], axis=1)
starbucks_age_missing_x.gender = np.where(starbucks_age_missing_x.gender=="M", 0, np.where(starbucks_age_missing_x.gender=="F", 1, 2))
starbucks_age_missing_x
#needs gender as number for prediction

In [ ]:
starbucks_age_missing
# not the same amount as in the previous part , may be that it was only a value of 118 that was too high, but 90 seems also too high

In [ ]:
starbucks_age_missing.age = extraReg.predict(starbucks_age_missing_x)
starbucks_imputed_full_v2 = starbucks_age_full.append(starbucks_age_missing)
starbucks_imputed_full_v2.age = round(starbucks_imputed_full_v2.age, 0)

In [ ]:
starbucks_imputed_full_v2.info()

In [ ]:
starbucks_imputed_full_v2.describe()

In [ ]:
starbucks_imputed_full_v2.to_csv("./starbucks_data/starbucks_imputed.csv", index=False)